In [2]:
import pandas as pd
import seaborn as sns
import numpy as np

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt #Visulization
from matplotlib import pyplot

matplotlib.rc('font',family = 'NanumGothicCoding')


from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection  import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error


from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.tree import export_graphviz
import graphviz

import statsmodels.formula.api as smf

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

###
from pandas import set_option
from scipy import stats

In [3]:
df_raw = pd.read_csv("./data/SCALE불량.csv", encoding = '949')
df_raw.head()

,PLATE_NO,ROLLING_DATE,SCALE,SPEC,STEEL_KIND,PT_THK,PT_WDTH,PT_LTH,PT_WGT,FUR_NO,...,FUR_HZ_TEMP,FUR_HZ_TIME,FUR_SZ_TEMP,FUR_SZ_TIME,FUR_TIME,FUR_EXTEMP,ROLLING_TEMP_T5,HSB,ROLLING_DESCALING,WORK_GR
0,PB562774,2008-08-01:00:00:15,양품,AB/EH32-TM,T1,32.25,3707,15109,14180,1호기,...,1144,116,1133,59,282,1133,934,적용,8,2조
1,PB562775,2008-08-01:00:00:16,양품,AB/EH32-TM,T1,32.25,3707,15109,14180,1호기,...,1144,122,1135,53,283,1135,937,적용,8,2조
2,PB562776,2008-08-01:00:00:59,양품,NV-E36-TM,T8,33.27,3619,19181,18130,2호기,...,1129,116,1121,55,282,1121,889,적용,8,3조
3,PB562777,2008-08-01:00:01:24,양품,NV-E36-TM,T8,33.27,3619,19181,18130,2호기,...,1152,125,1127,68,316,1127,885,적용,8,3조
4,PB562778,2008-08-01:00:01:44,양품,BV-EH36-TM,T8,38.33,3098,13334,12430,3호기,...,1140,134,1128,48,314,1128,873,적용,8,1조


In [4]:
df_raw.shape


(720, 21)

In [5]:
df_raw.dtypes

PLATE_NO              object
ROLLING_DATE          object
SCALE                 object
SPEC                  object
STEEL_KIND            object
PT_THK               float64
PT_WDTH                int64
PT_LTH                 int64
PT_WGT                 int64
FUR_NO                object
FUR_NO_ROW             int64
FUR_HZ_TEMP            int64
FUR_HZ_TIME            int64
FUR_SZ_TEMP            int64
FUR_SZ_TIME            int64
FUR_TIME               int64
FUR_EXTEMP             int64
ROLLING_TEMP_T5        int64
HSB                   object
ROLLING_DESCALING      int64
WORK_GR               object
dtype: object

In [6]:
df_raw.isnull().sum()

PLATE_NO             0
ROLLING_DATE         0
SCALE                0
SPEC                 0
STEEL_KIND           0
PT_THK               0
PT_WDTH              0
PT_LTH               0
PT_WGT               0
FUR_NO               0
FUR_NO_ROW           0
FUR_HZ_TEMP          0
FUR_HZ_TIME          0
FUR_SZ_TEMP          0
FUR_SZ_TIME          0
FUR_TIME             0
FUR_EXTEMP           0
ROLLING_TEMP_T5      0
HSB                  0
ROLLING_DESCALING    0
WORK_GR              0
dtype: int64

In [7]:
df_raw['SCALE'].describe()


count     720
unique      2
top        양품
freq      489
Name: SCALE, dtype: object

In [8]:
df_raw['SCALE'].count()

720

In [9]:
df_raw.describe()

,PT_THK,PT_WDTH,PT_LTH,PT_WGT,FUR_NO_ROW,FUR_HZ_TEMP,FUR_HZ_TIME,FUR_SZ_TEMP,FUR_SZ_TIME,FUR_TIME,FUR_EXTEMP,ROLLING_TEMP_T5,ROLLING_DESCALING
count,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000
mean,29.267857,2807.127778,34323.269444,41481.777778,1.497222,1155.338889,88.527778,1149.666667,78.802778,300.563889,1149.666667,933.920833,8.319444
std,19.152651,517.839690,13828.434837,24751.322015,0.500340,20.866398,37.660799,17.571788,36.421004,45.390303,17.571788,107.863887,1.623252
min,12.030000,1800.000000,7893.000000,7820.000000,1.000000,1103.000000,43.000000,1113.000000,35.000000,221.000000,1113.000000,0.000000,5.000000
25%,16.090000,2466.000000,22663.000000,17757.500000,1.000000,1139.000000,65.000000,1132.000000,57.750000,257.000000,1132.000000,889.750000,6.000000
50%,20.130000,2725.000000,37380.000000,36120.000000,1.000000,1158.000000,75.000000,1155.000000,67.500000,304.000000,1155.000000,951.000000,8.000000
75%,40.000000,3125.000000,43735.000000,61120.000000,2.000000,1171.000000,101.000000,1164.000000,86.000000,342.000000,1164.000000,994.250000,10.000000
max,100.340000,4575.000000,54917.000000,115080.000000,2.000000,1206.000000,312.000000,1185.000000,294.000000,398.000000,1185.000000,1078.000000,10.000000


In [10]:
# sns.pairplot(df_raw, diag_kind='hist')
# plt.show()

In [12]:
# sns.pairplot(df_raw,
#             diag_kind = 'kde',
#             hue = 'SCALE',
#             palette='bright')
# plt.show()

In [13]:
df_raw.head()

,PLATE_NO,ROLLING_DATE,SCALE,SPEC,STEEL_KIND,PT_THK,PT_WDTH,PT_LTH,PT_WGT,FUR_NO,...,FUR_HZ_TEMP,FUR_HZ_TIME,FUR_SZ_TEMP,FUR_SZ_TIME,FUR_TIME,FUR_EXTEMP,ROLLING_TEMP_T5,HSB,ROLLING_DESCALING,WORK_GR
0,PB562774,2008-08-01:00:00:15,양품,AB/EH32-TM,T1,32.25,3707,15109,14180,1호기,...,1144,116,1133,59,282,1133,934,적용,8,2조
1,PB562775,2008-08-01:00:00:16,양품,AB/EH32-TM,T1,32.25,3707,15109,14180,1호기,...,1144,122,1135,53,283,1135,937,적용,8,2조
2,PB562776,2008-08-01:00:00:59,양품,NV-E36-TM,T8,33.27,3619,19181,18130,2호기,...,1129,116,1121,55,282,1121,889,적용,8,3조
3,PB562777,2008-08-01:00:01:24,양품,NV-E36-TM,T8,33.27,3619,19181,18130,2호기,...,1152,125,1127,68,316,1127,885,적용,8,3조
4,PB562778,2008-08-01:00:01:44,양품,BV-EH36-TM,T8,38.33,3098,13334,12430,3호기,...,1140,134,1128,48,314,1128,873,적용,8,1조


In [14]:
df_raw.dtypes

PLATE_NO              object
ROLLING_DATE          object
SCALE                 object
SPEC                  object
STEEL_KIND            object
PT_THK               float64
PT_WDTH                int64
PT_LTH                 int64
PT_WGT                 int64
FUR_NO                object
FUR_NO_ROW             int64
FUR_HZ_TEMP            int64
FUR_HZ_TIME            int64
FUR_SZ_TEMP            int64
FUR_SZ_TIME            int64
FUR_TIME               int64
FUR_EXTEMP             int64
ROLLING_TEMP_T5        int64
HSB                   object
ROLLING_DESCALING      int64
WORK_GR               object
dtype: object

In [15]:
df_raw['PLATE_NO'].unique()
df_raw['STEEL_KIND'].unique()
df_raw['SPEC'].unique()
df_raw['FUR_NO'].unique()
df_raw['HSB'].unique()
df_raw['WORK_GR'].unique()
df_raw['ROLLING_DATE'].unique()


array(['2008-08-01:00:00:15', '2008-08-01:00:00:16',
       '2008-08-01:00:00:59', '2008-08-01:00:01:24',
       '2008-08-01:00:01:44', '2008-08-01:00:02:06',
       '2008-08-01:00:02:28', '2008-08-01:00:02:21',
       '2008-08-01:00:02:51', '2008-08-01:00:03:15',
       '2008-08-01:00:03:24', '2008-08-01:00:04:15',
       '2008-08-01:00:04:20', '2008-08-01:00:05:47',
       '2008-08-01:00:05:25', '2008-08-01:00:05:16',
       '2008-08-01:01:10:14', '2008-08-01:01:10:44',
       '2008-08-01:01:11:01', '2008-08-01:01:11:08',
       '2008-08-01:01:12:45', '2008-08-01:01:12:49',
       '2008-08-01:01:13:47', '2008-08-01:01:13:05',
       '2008-08-01:01:14:20', '2008-08-01:01:14:53',
       '2008-08-01:01:14:25', '2008-08-01:01:15:39',
       '2008-08-01:01:15:14', '2008-08-01:01:15:59',
       '2008-08-01:01:15:34', '2008-08-01:02:20:52',
       '2008-08-01:02:20:28', '2008-08-01:02:21:27',
       '2008-08-01:02:21:11', '2008-08-01:02:21:37',
       '2008-08-01:02:21:17', '2008-08-01:02:2

In [16]:
df_raw_y = df_raw['SCALE']
df_raw_x = df_raw.drop(["SCALE",'SPEC','PLATE_NO','STEEL_KIND','ROLLING_DATE'], axis = 1, inplace = False)
df_raw_x.head()

,PT_THK,PT_WDTH,PT_LTH,PT_WGT,FUR_NO,FUR_NO_ROW,FUR_HZ_TEMP,FUR_HZ_TIME,FUR_SZ_TEMP,FUR_SZ_TIME,FUR_TIME,FUR_EXTEMP,ROLLING_TEMP_T5,HSB,ROLLING_DESCALING,WORK_GR
0,32.25,3707,15109,14180,1호기,1,1144,116,1133,59,282,1133,934,적용,8,2조
1,32.25,3707,15109,14180,1호기,2,1144,122,1135,53,283,1135,937,적용,8,2조
2,33.27,3619,19181,18130,2호기,1,1129,116,1121,55,282,1121,889,적용,8,3조
3,33.27,3619,19181,18130,2호기,2,1152,125,1127,68,316,1127,885,적용,8,3조
4,38.33,3098,13334,12430,3호기,1,1140,134,1128,48,314,1128,873,적용,8,1조


In [17]:
df_raw_dummy = pd.get_dummies(df_raw_x)
df_raw_dummy.head()

,PT_THK,PT_WDTH,PT_LTH,PT_WGT,FUR_NO_ROW,FUR_HZ_TEMP,FUR_HZ_TIME,FUR_SZ_TEMP,FUR_SZ_TIME,FUR_TIME,...,ROLLING_DESCALING,FUR_NO_1호기,FUR_NO_2호기,FUR_NO_3호기,HSB_미적용,HSB_적용,WORK_GR_1조,WORK_GR_2조,WORK_GR_3조,WORK_GR_4조
0,32.25,3707,15109,14180,1,1144,116,1133,59,282,...,8,1,0,0,0,1,0,1,0,0
1,32.25,3707,15109,14180,2,1144,122,1135,53,283,...,8,1,0,0,0,1,0,1,0,0
2,33.27,3619,19181,18130,1,1129,116,1121,55,282,...,8,0,1,0,0,1,0,0,1,0
3,33.27,3619,19181,18130,2,1152,125,1127,68,316,...,8,0,1,0,0,1,0,0,1,0
4,38.33,3098,13334,12430,1,1140,134,1128,48,314,...,8,0,0,1,0,1,1,0,0,0


In [23]:
df_raw_y = df_raw['SCALE']
df_raw_y = pd.get_dummies(df_raw_y)
df_raw_y.head()

,불량,양품
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [19]:
df_raw_x = df_raw_dummy
df_raw_x.head()

,PT_THK,PT_WDTH,PT_LTH,PT_WGT,FUR_NO_ROW,FUR_HZ_TEMP,FUR_HZ_TIME,FUR_SZ_TEMP,FUR_SZ_TIME,FUR_TIME,...,ROLLING_DESCALING,FUR_NO_1호기,FUR_NO_2호기,FUR_NO_3호기,HSB_미적용,HSB_적용,WORK_GR_1조,WORK_GR_2조,WORK_GR_3조,WORK_GR_4조
0,32.25,3707,15109,14180,1,1144,116,1133,59,282,...,8,1,0,0,0,1,0,1,0,0
1,32.25,3707,15109,14180,2,1144,122,1135,53,283,...,8,1,0,0,0,1,0,1,0,0
2,33.27,3619,19181,18130,1,1129,116,1121,55,282,...,8,0,1,0,0,1,0,0,1,0
3,33.27,3619,19181,18130,2,1152,125,1127,68,316,...,8,0,1,0,0,1,0,0,1,0
4,38.33,3098,13334,12430,1,1140,134,1128,48,314,...,8,0,0,1,0,1,1,0,0,0


In [24]:
df_raw.shape

(720, 21)

In [20]:
df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(df_raw_x, df_raw_y
                                                               , test_size = 0.3, random_state = 1234)
print('train data X size : {}'.format(df_train_x.shape))
print('train data Y size : {}'.format(df_train_y.shape))
print('test data X size : {}'.format(df_test_x.shape))
print('test data Y size : {}'.format(df_test_y.shape))


train data X size : (504, 22)
train data Y size : (504, 2)
test data X size : (216, 22)
test data Y size : (216, 2)


In [21]:
lr = LinearRegression()
lr.fit(df_train_x, df_train_y)
pred_x = lr.predict(df_test_x)

print('<LinearRegression>')
print('Score on trainning set : {:.3f}'.format(lr.score(df_train_x, df_train_y)))
print('Score on test set : {:.3f}'.format(lr.score(df_test_x, df_test_y)))
print('mse : {:.3f}'.format(mean_squared_error(df_test_y, pred_x)))
print('mae : {:.3f}'.format(mean_absolute_error(df_test_y, pred_x)))

<LinearRegression>
Score on trainning set : 0.483
Score on test set : 0.446
mse : 0.122
mae : 0.291
